# 1. Basic Analysis

## Libraries

In [ ]:
from basic_analysis import create_graph, add_edges, draw_graph, graph_data
from basic_analysis import degree_distribution, in_out_degree_distributions
from basic_analysis import dialogue_distribution, loglog_degree_distributions
from basic_analysis import fit_powerlaw
from basic_analysis import centrality_analysis, assortativity_analysis

from sentiment_analysis import sentiment_heatmap, top_relationships, sentiment_network, most_loved_hated, plot_heatmap

from indirect_analysis import create_graph_from_matrix

## Get data

In [ ]:
all_dialogues, G = create_graph(path_name='data/dialogues.csv')
all_dialogues, G = add_edges(G, path_name='data/dialogues.csv', edge_type='direct')

## Draw Graph

In [ ]:
draw_graph(G)

## Print basic graph data

In [ ]:
graph_data(G)

## Degree distribution

In [ ]:
total_degrees = degree_distribution(G)
in_degrees, out_degrees = in_out_degree_distributions(G)

In [ ]:
dialogue_counts = dialogue_distribution(G)

## Try Fit Powerlaw

In [ ]:

print("POWER-LAW EXPONENTS (α) AND FIT RANGE")

alpha_in, x_min_in = fit_powerlaw(in_degrees)
print(f"In-Degree α = {alpha_in:.3f}  (xmin = {x_min_in})")

alpha_out, x_min_out = fit_powerlaw(out_degrees)
print(f"Out-Degree α = {alpha_out:.3f}  (xmin = {x_min_out})")

alpha_total, x_min_total = fit_powerlaw(total_degrees)
print(f"Total-Degree α = {alpha_total:.3f}  (xmin = {x_min_total})")

alpha_dialogue, x_min_dialogue = fit_powerlaw(dialogue_counts)
print(f"Dialogue-Count α = {alpha_dialogue:.3f}  (xmin = {x_min_dialogue})")


In [ ]:
_ = loglog_degree_distributions(G)

## Centrality and Assortativity

In [ ]:
_ = centrality_analysis(G)

In [ ]:
assortativity_analysis(G)

- - -

# 2. Sentiment Analysis


In [ ]:
G, dialogue_data, relationship_data = sentiment_heatmap(G, remove_empty=True)

In [ ]:
_ = top_relationships(relationship_data, n=10)

In [ ]:
_ = sentiment_network(G, relationship_data)

# Most loved and hated characters (with Bayesian average)

In [ ]:
most_loved, most_hated = most_loved_hated(relationship_data, n=10)

print("=" * 60)
print("MOST LOVED CHARACTERS")
for i, loved in enumerate(most_loved):
    print(f"{i+1}. {loved['character']}: {loved['charisma']:.3f} on {loved['incoming_dialogues']} dialogues")

print("\n" + "=" * 60)
print("MOST HATED CHARACTERS")
for i, hated in enumerate(most_hated):
    print(f"{i+1}. {hated['character']}: {hated['charisma']:.3f} on {hated['incoming_dialogues']} dialogues")

In [ ]:
about_dialogues, AG = create_graph()
about_dialogues, AG = add_edges(AG, path_name="data/dialogues.csv", edge_type="indirect")

In [ ]:
spring_layout_args = {
    'k': 2.5,
    'iterations': 250,
    'seed': 42
}
draw_graph(AG, spring_layout_args)

In [ ]:
AG, about_dialogue_data, about_relationship_data = sentiment_heatmap(AG, remove_empty=True)

In [ ]:
most_loved, most_hated = most_loved_hated(about_relationship_data, n=10)

print("=" * 60)
print("MOST LOVED CHARACTERS")
for i, loved in enumerate(most_loved):
    print(f"{i+1}. {loved['character']}: {loved['charisma']:.3f} on {loved['incoming_dialogues']} dialogues")

print("\n" + "=" * 60)
print("MOST HATED CHARACTERS")
for i, hated in enumerate(most_hated):
    print(f"{i+1}. {hated['character']}: {hated['charisma']:.3f} on {hated['incoming_dialogues']} dialogues")

In [ ]:
import numpy as np

# compute the element-wise difference between the two relationship_data dicts of dict:
difference_matrix = []
all_characters = []

for speaker in relationship_data:
    difference_matrix.append([])
    all_characters.append(speaker)
    for target in relationship_data[speaker]:
        # Safely get values using .get() to handle missing keys
        about_item = about_relationship_data.get(speaker, {}).get(target)
        direct_item = relationship_data[speaker][target]
        
        a = about_item['adjusted_sentiment'] if about_item is not None else None
        b = direct_item['adjusted_sentiment'] if direct_item is not None else None
        
        difference_matrix[-1].append(np.nan if a is None or b is None else a - b)

# Convert list of lists to numpy array
difference_matrix = np.array(difference_matrix)

difference_matrix, col_characters, row_characters = plot_heatmap(difference_matrix, all_characters, remove_empty=True)

In [ ]:
G = create_graph_from_matrix(difference_matrix, row_characters, col_characters)
draw_graph(G, color_parameter='sentiment')